### Project 2. 
#### Checking partial derivatives of descent gradient

#### Catalina Jaramillo
#### Angélica Narvaes

In [1]:
import numpy as np
import pandas as pd
import copy

In [2]:
# sigmoid function

def sigmoid(z):
    return 1/(1+np.exp(-z))

In [3]:
# Derivative of sigmoid function

def sigmoidGradient(z):
    sigmoid = 1/(1+np.exp(-z))
    return sigmoid*(1-sigmoid)

In [4]:
# theta as a list

def listTheta(thetai, input_layer_size, hidden_layer_size, num_labels):
    
    theta = []
    
    aux = 0
     
    # if there is only 1 hidden layer
    if type(hidden_layer_size)==int:
        theta.append(thetai[:((input_layer_size+1)*hidden_layer_size)].reshape(hidden_layer_size, input_layer_size+1))
        theta.append(thetai[((input_layer_size+1)*hidden_layer_size):].reshape(num_labels, hidden_layer_size+1))
        return theta
    else:   
        l=len(hidden_layer_size)
        
    # if there is more than 1 hidden layer    
    for i in range(l+1):
        if i==0:
            aux = (input_layer_size+1)*hidden_layer_size[i]
            theta.append(thetai[:aux].reshape(hidden_layer_size[i], input_layer_size+1))
        elif i==l:
            aux1= copy.deepcopy(aux)
            aux = aux + (hidden_layer_size[i-1]+1)*num_labels
            theta.append(thetai[aux1:aux].reshape(num_labels, hidden_layer_size[i-1]+1))
        elif i>0 and i<l:
            aux1= copy.deepcopy(aux)
            aux = aux + (hidden_layer_size[i-1]+1)*(hidden_layer_size[i])
            theta.append(thetai[aux1:aux].reshape(hidden_layer_size[i], hidden_layer_size[i-1]+1))

    return theta

In [5]:
# theta list as a flatten vector

def flattenTheta(thetai, input_layer_size, hidden_layer_size, num_labels):
    for i in range(len(thetai)):
        if i==0:
            theta = thetai[i].flatten()
        else:
            theta = np.insert(theta, len(theta), thetai[i].flatten())
            
    
    return theta


In [6]:
# number of elements of layer size (camada de entrada)
# number of elements of hidden layer
# number of elements of output: num_labels

def computeCost(thetai,X,y,input_layer_size, hidden_layer_size, num_labels, Lambda):
                   
    theta = listTheta(thetai, input_layer_size, hidden_layer_size, num_labels)
    l=len(theta)
    
    m = X.shape[0]
    # number of layers
    
    J = 0
    
    #join the vector of 1s
    
    X = np.hstack((np.ones([m,1]),X))
    y10 = np.zeros([m,num_labels])
    
    #Activation list
    a=[]

    for i in range(l):
        #1st activation
        if i==0:
            a1 = sigmoid(X @ theta[i].T)
            a1 = np.hstack((np.ones([m,1]),a1))
            a.append(a1)
       
        #final activation
        elif i==l-1:
            a2 = sigmoid(a[i-1] @ theta[i].T)
            a.append(a2)
            
        #hidden activation
        else:
            a1 = sigmoid(a[i-1] @ theta[i].T)
            a1 = np.hstack((np.ones([m,1]),a1))
            a.append(a1)

                
    for i in range(1,num_labels+1):
        #creates the matrix of ys, where each column is the vector of label, with 1 in position i if class=i
    
        ''' np.newaxis might come in handy when you want to explicitly convert a 1D array to 
            either a row vector or a column vector.
        '''
        y10[:,i-1][:,np.newaxis] = np.where(y==i,1,0)
        
    for j in range(num_labels):
        #adding all log cost function
        J = J - np.sum(y10[:,j]*np.log(a[l-1][:,j])+(1-y10[:,j])*np.log(1-a[l-1][:,j]))
        
    cost = (1/m)*J
    
    #regularized cost for logistic regression
    regj = 0
    # derivatives for each layer  
    grad = []
    for i in range(l):
        regj = regj+np.sum(theta[i][:,1:]**2)   ####CHECK IF IT CAN BE CHANGED TO DOT PRODUCT.
        grad.append(np.zeros((theta[i].shape)))
      
    reg_J = cost + Lambda/(2*m)*(regj)
            
    #computing update of thetas
    for i in range(m):
        xi = X[i,:]
        
        #activation i
        ai= []
        ai.append(xi)
        
        for j in range(l):
            ai.append(a[j][i,:])
       
        # deltas 
        delta = []
        #len(ai)=l+1
        for j in range(l,0,-1):
            # delta in output
            if j==l:
                # 1st delta
                delta.append(ai[j]-y10[i,:])
            
            #delta hidden layer
            
            #the case of layer L-1 is different because of bias on L-1 and not bias on output
            elif j==l-1:
                delta.append((theta[j].T @ delta[l-j-1])*(ai[j])*(1-ai[j]))
            
            else: 
                #never delta(0)
                delta.append((theta[j].T @ delta[l-j-1][1:])*(ai[j])*(1-ai[j]))
        
        # reverse delta because it's created by backpropagation
        delta.reverse()
     
        # grad = grad anterior(layer l) + delta(layer l+1)*(a(layer l))^T 
        
        #update grad
        
        for j in range(l):
            # recall ai[j] = a[j-1]... len(ai)=l+1 because a1[0]=xi
            if j==l-1:
                grad[j] = grad[j] +  delta[j].T[:,np.newaxis]@ai[j][:,np.newaxis].T
            else:
                grad[j] = grad[j] + delta[j][1:][:,np.newaxis]@ ai[j][:,np.newaxis].T

    grad_reg = []

    for i in range(l):
        grad[i] = (1/m)*grad[i]
        grad_reg.append(grad[i]+(Lambda/m)*np.hstack((np.zeros([theta[i].shape[0],1]), theta[i][:,1:])))
    

    # flatten grad_red
    for i in range (len(grad_reg)):
        if i==0:
            g1 = grad_reg[i].flatten()
        else:
            g1 = np.insert(g1, len(g1), grad_reg[i].flatten())
    
    return reg_J,g1

In [7]:
#initialize weights

def randInitializeWeights(L_in,L_out):
    epi = (2**(1/2))/(L_in+L_out)**(1/2)
    # Recall the matrix is like in the other sense and we need to add the bais weight
    W = np.random.rand(L_out,L_in+1)*(2*epi)-epi
    return W

In [8]:
# creates the initial theta according to initial layer size, hidden layer size and number of labels

def initialTheta(input_layer_size, hidden_layer_size,num_labels):
    theta = []
    if type(hidden_layer_size)== int:
        theta.append(randInitializeWeights(input_layer_size,hidden_layer_size))
        theta.append(randInitializeWeights(hidden_layer_size,num_labels))
    else:
        k = len(hidden_layer_size)
        for i in range(k+1):
            if i==0:
                theta.append(randInitializeWeights(input_layer_size, hidden_layer_size[0]))
            elif i== k:
                theta.append(randInitializeWeights(hidden_layer_size[k-1], num_labels))
            else:
                theta.append(randInitializeWeights(hidden_layer_size[i-1],hidden_layer_size[i]))
    return theta

In [9]:
# n_input: length of each example
# n_output: number of classes
def randomData(n_examples, n_input, n_output):
    X = np.random.rand(n_examples,n_input)
    y = np.random.randint(n_output, size=n_examples)+1
    return X,y

### Secant line

In [11]:
def secantCost(X,y,theta,input_layer_size, hidden_layer_size, num_labels, epsilon, Lambda):
    J =[]
    for i in range(len(theta)):
        for j in range(len(theta[i])):
            for k in range(len(theta[i][j])):
                theta1 = copy.deepcopy(theta)
                theta2 = copy.deepcopy(theta)
                theta1[i][j][k] = theta[i][j][k]+epsilon
                theta2[i][j][k] = theta[i][j][k]-epsilon
#                 J1 = computeCost(theta1, X,y.reshape(n_examples,1),input_layer_size, hidden_layer_size, num_labels, Lambda)
#                 J2 = computeCost(theta2,X,y.reshape(n_examples,1),input_layer_size, hidden_layer_size, num_labels, Lambda)
                J1 = computeCost(flattenTheta(theta1, input_layer_size, hidden_layer_size, num_labels), X,y.reshape(n_examples,1),input_layer_size, hidden_layer_size, num_labels, Lambda)
                J2 = computeCost(flattenTheta(theta2, input_layer_size, hidden_layer_size, num_labels),X,y.reshape(n_examples,1),input_layer_size, hidden_layer_size, num_labels, Lambda)
                J.append((J1[0]-J2[0])/(2*epsilon))
    return J
        

In [ ]:
epsilon = 0.0001
Lambda = 1
secant= secantCost(X,y.reshape(n_examples,1),initial_theta,input_layer_size, hidden_layer_size, num_labels, epsilon, Lambda)

### Comparison between secant line and derivative

In [18]:
# check if the norm2 of difference between gradient and secant line is smaller than tolerance

def checking(grad, sec, tolerance):
    # we need flatten grad
    
    if np.linalg.norm(flattenTheta(grad, input_layer_size, hidden_layer_size, num_labels)-secant) <= tolerance:
        return True, np.linalg.norm(flattenTheta(grad, input_layer_size, hidden_layer_size, num_labels)-secant)
    else:
        return False, np.linalg.norm(flattenTheta(grad, input_layer_size, hidden_layer_size, num_labels)-secant)
    

In [20]:
## Initialize

n_examples=3
input_layer_size = 2

# list of hidden layers'size
hidden_layer_size = 5
num_labels = 4

initial_theta = initialTheta(input_layer_size,hidden_layer_size,num_labels)
initial_theta_flatten = flattenTheta(initial_theta, input_layer_size, hidden_layer_size, num_labels)

X,y = randomData(n_examples, input_layer_size, num_labels)

cost, grad = computeCost(initial_theta_flatten, X, y.reshape(n_examples,1),input_layer_size,hidden_layer_size,num_labels,0)


In [25]:
# True if the norm 2 to subtract secant line and gradient is smaller than tolerance 

tolerance = 0.000000001
epsilon = 0.0001

# should be non regularized
Lambda = 0
secant= secantCost(X,y.reshape(n_examples,1),initial_theta,input_layer_size, hidden_layer_size, num_labels, epsilon, Lambda)



In [26]:
checking(grad, secant,tolerance)

(True, 2.806268387653222e-10)